In [1]:
#Import library
import numpy as np
import random as rm
import pandas as pd
from matplotlib import pyplot as plt, cm
import sklearn as sc
import seaborn as sns
import os
import sys
import json
import matplotlib
import re
import csv
import shutil
import math
from numpy.lib.shape_base import tile
from sklearn.preprocessing import normalize
import string
import re
from collections import defaultdict
from difflib import SequenceMatcher
import markov_clustering as mc
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph
import imageio
import requests
import urllib.request

# Reading and preprocessing the raw data

In [2]:
def create_df(file):
  '''
  Creates pandas dataframe from raw json file. 
  @ file: file path to raw json file.
  @ return: returns dataframe
  '''
  # dates = ['2024-08-30','2024-08-31','2024-09-01','2024-09-02','2024-09-03','2024-09-04','2024-09-05',
  #          '2024-09-06','2024-09-07','2024-09-08','2024-09-09','2024-09-10','2024-09-17','2024-09-18','2024-09-19',
  #          '2024-09-20','2024-09-21','2024-09-22','2024-09-23','2024-09-24','2024-09-25','2024-09-26']
  dates=['2024-10-02','2024-10-03','2024-10-04','2024-10-08','2024-10-09','2024-10-10',
         '2024-10-11','2024-10-12','2024-10-13','2024-10-14','2024-10-15']
  #df_pending = pd.read_json(file, lines=True)
  # concatenate all 30 day's logs
  df_list = []

  df_temp = df_temp = pd.read_json(f'{file}.2024-10-01', lines=True)
  df_list.append(df_temp)

  for d in dates:
    file2 = f'{file}.{d}'
    print("doing", file2)
    try:
      df_temp = pd.read_json(file2, lines=True)
      df_list.append(df_temp)
    except:
      print("Error Occured (no need to care)")

  df_pending = pd.concat(df_list)
  df_pending['timestamp'] = pd.to_datetime(df_pending['timestamp'])
  #df_pending = df_pending[df_pending.sourceIp != "128.59.15.19"]

  df_pending.sort_values(by='timestamp', inplace = True)
  df_pending.drop(df_pending[df_pending['src_port'] == 0].index, inplace=True)
  df_pending.index = range(len(df_pending))
  return df_pending

There are 3 honeypots in total - I named them "pending", "ebilling", and "csh". <br>
One dataframe is created for each honeypots, so 3 dataframes are created in total: `raw_csh` `raw_pending` `raw_ebilling`

In [ ]:
# takes about 10 sec
raw_pending = create_df('./raw_data/test_raw_data/pending/cowrie.json')

raw_ebilling = create_df('./raw_data/test_raw_data/ebilling/cowrie.json')

doing ./test_data/pending/cowrie.json.2024-10-02
doing ./test_data/pending/cowrie.json.2024-10-03
doing ./test_data/pending/cowrie.json.2024-10-04
doing ./test_data/pending/cowrie.json.2024-10-08
doing ./test_data/pending/cowrie.json.2024-10-09
doing ./test_data/pending/cowrie.json.2024-10-10
doing ./test_data/pending/cowrie.json.2024-10-11
doing ./test_data/pending/cowrie.json.2024-10-12
doing ./test_data/pending/cowrie.json.2024-10-13
doing ./test_data/pending/cowrie.json.2024-10-14
doing ./test_data/pending/cowrie.json.2024-10-15
doing ./test_data/ebilling/cowrie.json.2024-10-02
doing ./test_data/ebilling/cowrie.json.2024-10-03
doing ./test_data/ebilling/cowrie.json.2024-10-04
doing ./test_data/ebilling/cowrie.json.2024-10-08
doing ./test_data/ebilling/cowrie.json.2024-10-09
doing ./test_data/ebilling/cowrie.json.2024-10-10
doing ./test_data/ebilling/cowrie.json.2024-10-11
doing ./test_data/ebilling/cowrie.json.2024-10-12
doing ./test_data/ebilling/cowrie.json.2024-10-13
doing ./tes

In [ ]:
raw_csh = create_df('./raw_data/test_raw_data/csh/cowrie.json')

doing ./test_data/csh/cowrie.json.2024-10-02
doing ./test_data/csh/cowrie.json.2024-10-03
doing ./test_data/csh/cowrie.json.2024-10-04
doing ./test_data/csh/cowrie.json.2024-10-08
doing ./test_data/csh/cowrie.json.2024-10-09
doing ./test_data/csh/cowrie.json.2024-10-10
doing ./test_data/csh/cowrie.json.2024-10-11
doing ./test_data/csh/cowrie.json.2024-10-12
doing ./test_data/csh/cowrie.json.2024-10-13
doing ./test_data/csh/cowrie.json.2024-10-14
doing ./test_data/csh/cowrie.json.2024-10-15


In [8]:
# example of raw dataframe.
raw_ebilling

,eventid,src_ip,src_port,dst_ip,dst_port,session,protocol,message,sensor,timestamp,...,outfile,destfile,fingerprint,key,type,data,id,filename,name,value
0,cowrie.session.connect,41.223.231.182,52520.0,192.168.144.2,22.0,c54c55c11ce6,ssh,New connection: 41.223.231.182:52520 (192.168....,2f38450f0164,2024-10-01 17:37:19.053895+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cowrie.client.version,41.223.231.182,NaN,NaN,NaN,c54c55c11ce6,NaN,Remote SSH version: SSH-2.0-Go,2f38450f0164,2024-10-01 17:37:19.058676+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cowrie.client.kex,41.223.231.182,NaN,NaN,NaN,c54c55c11ce6,NaN,SSH client hassh fingerprint: 0a07365cc01fa9fc...,2f38450f0164,2024-10-01 17:37:19.301097+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cowrie.login.failed,41.223.231.182,NaN,NaN,NaN,c54c55c11ce6,NaN,login attempt [daniel/daniel] failed,2f38450f0164,2024-10-01 17:37:20.029641+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cowrie.session.closed,41.223.231.182,NaN,NaN,NaN,c54c55c11ce6,NaN,Connection lost after 2 seconds,2f38450f0164,2024-10-01 17:37:21.276602+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259508,cowrie.client.version,47.187.9.153,NaN,NaN,NaN,8f0fd7dfdfd6,NaN,Remote SSH version: SSH-2.0-OpenSSH_7.4,3dcf412fea67,2024-10-15 02:44:43.601357+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259509,cowrie.client.kex,47.187.9.153,NaN,NaN,NaN,8f0fd7dfdfd6,NaN,SSH client hassh fingerprint: acaa53e0a7d7ac7d...,3dcf412fea67,2024-10-15 02:44:43.602326+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259510,cowrie.login.failed,47.187.9.153,NaN,NaN,NaN,8f0fd7dfdfd6,NaN,login attempt [mail/987654321] failed,3dcf412fea67,2024-10-15 02:44:44.530775+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259511,cowrie.login.failed,47.187.9.153,NaN,NaN,NaN,8f0fd7dfdfd6,NaN,login attempt [mail/987654321] failed,3dcf412fea67,2024-10-15 02:44:46.122572+00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
set(raw_pending['eventid'])

{'cowrie.client.fingerprint',
 'cowrie.client.kex',
 'cowrie.client.var',
 'cowrie.client.version',
 'cowrie.command.failed',
 'cowrie.command.input',
 'cowrie.command.success',
 'cowrie.direct-tcpip.data',
 'cowrie.direct-tcpip.request',
 'cowrie.log.closed',
 'cowrie.login.failed',
 'cowrie.login.success',
 'cowrie.session.closed',
 'cowrie.session.connect',
 'cowrie.session.file_download',
 'cowrie.session.file_upload',
 'cowrie.session.params'}

In [10]:
def create_clean_df(df):
  '''
  Takes a raw dataframe and deletes unnecessary columns.
  @ df: pandas dataframe (the raw dataframe returned by create_df)
  @ return: cleaned dataframe
  '''
  df_clean = pd.DataFrame()
  list_tf = list()
  list_ip=list()
  list_port = list()
  list_node = list()
  list_session = list()

  for i in range(len(df)):
    if df['eventid'][i] =="cowrie.command.input":
      cmd = df['message'][i][5:]
      if cmd=='':
        continue

      list_tf.append(df['timestamp'][i])
      list_ip.append(df['src_ip'][i])
      list_port.append(df['src_port'][i])
      list_session.append(df['session'][i])

      list_node.append(cmd)
    # else:
    # #   if df['eventid'][i]=="cowrie.session.connect":
    #     list_tf.append(df['timestamp'][i])
    #     list_ip.append(df['src_ip'][i])
    #     list_port.append(df['src_port'][i])
    #     list_session.append(df['session'][i])
    #     list_node.append(df['eventid'][i])

  '''
    if len(df['session'][i]) ==0:
      continue
    if len(df['session'][i]) ==1:
      if(df['session'][i][0][0]=="out"):
        continue
    if(df['session'][i][0][0]=="out"):
      df['session'][i].pop(0)

    list_ses.append(df['session'][i])
    list_protocol.append(df['protocol'][i])
    list_ip.append(df['src_ip'][i])
  '''
  df_clean["timestamp"] = list_tf
  df_clean["src_ip"] = list_ip
  df_clean["src_port"] = list_port
  df_clean["session"] = list_session

  df_clean["node"] = list_node


  return df_clean

In [11]:
# takes about 35 secs
# cleans each raw dataframes
pending_clean = create_clean_df(raw_pending)
ebilling_clean = create_clean_df(raw_ebilling)
csh_clean = create_clean_df(raw_csh)

# example of cleaned dataframe. "node" column is the one you want to cluster.
pending_clean

,timestamp,src_ip,src_port,session,node
0,2024-10-01 17:38:50.255261+00:00,172.245.223.99,NaN,6db2921c54ca,cd ~; chattr -ia .ssh; lockr -ia .ssh
1,2024-10-01 17:38:50.647108+00:00,172.245.223.99,NaN,6db2921c54ca,"cd ~ && rm -rf .ssh && mkdir .ssh && echo ""ssh..."
2,2024-10-01 17:38:54.128272+00:00,172.245.223.99,NaN,6db2921c54ca,cat /proc/cpuinfo | grep name | wc -l
3,2024-10-01 17:38:54.435426+00:00,172.245.223.99,NaN,6db2921c54ca,"echo ""root:zCmhdtIXoB9z""|chpasswd|bash"
4,2024-10-01 17:38:54.813275+00:00,172.245.223.99,NaN,6db2921c54ca,rm -rf /tmp/secure.sh; rm -rf /tmp/auth.sh; pk...
...,...,...,...,...,...
61133,2024-10-15 02:44:37.043298+00:00,175.194.54.19,NaN,6899a8110b8b,enable
61134,2024-10-15 02:44:37.045261+00:00,175.194.54.19,NaN,6899a8110b8b,system
61135,2024-10-15 02:44:37.047321+00:00,175.194.54.19,NaN,6899a8110b8b,ping; sh
61136,2024-10-15 02:44:37.661395+00:00,175.194.54.19,NaN,6899a8110b8b,/bin/busybox cat /proc/self/exe || cat /proc/s...


# Clustering nodes

In [ ]:
df_conversion = pd.read_csv('command_to_description.csv')

In [18]:
conversion_dict = dict()
for i in range(len(df_conversion)):
    #print(df_conversion["Node"][i])
    temp = eval(df_conversion["Node"][i])

    for node in temp:
        conversion_dict[node] = df_conversion["Job"][i]
print(conversion_dict)


{'cd ~; chattr -ia .ssh; lockr -ia .ssh': 'Lock SSH directory', 'cd ~ && rm -rf .ssh && mkdir .ssh && echo "ssh-rsa AAAAB3NzaC1yc2EAAAABJQAAAQEArDp4cun2lhr4KUhBGE7VvAcwdli2a8dbnrTOrbMz1+5O73fcBOx8NVbUT0bUanUV9tJ2/9p7+vD0EpZ3Tz/+0kX34uAx1RV/75GVOmNx+9EuWOnvNoaJe0QXxziIg9eLBHpgLMuakb5+BgTFB+rKJAw9u9FSTDengvS8hX1kNFS4Mjux0hJOK8rvcEmPecjdySYMb66nylAKGwCEE6WEQHmd1mUPgHwGQ0hWCwsQk13yCGPK5w6hYp5zYkFnvlC8hGmd4Ww+u97k6pfTGTUbJk14ujvcD9iUKQTTWYYjIIu5PmUux5bsZ0R4WFwdIe6+i6rBLAsPKgAySVKPRK+oRw== mdrfckr">>.ssh/authorized_keys && chmod -R go= ~/.ssh && cd ~': 'Save SSH key', 'cat /proc/cpuinfo | grep name | wc -l': 'Get CPU counts', 'echo "root:xb8SuHdl7brF"|chpasswd|bash': 'change password', 'echo "root:tyUGmOPT3PXP"|chpasswd|bash': 'change password', 'echo "root:cwcrJeXZ7DaS"|chpasswd|bash': 'change password', 'echo "root:qWPhXXXTD2z0"|chpasswd|bash': 'change password', 'echo "root:g2bEV3nauiFb"|chpasswd|bash': 'change password', 'echo "root:CK0Nhb6BcSTx"|chpasswd|bash': 'change password', 'echo 

In [19]:
def assign_description(df):
    list_description=list()
    for i in range(len(df)):
        list_description.append(conversion_dict[df["node"][i]])
    df["description"]=list_description
    return df


In [20]:
pending_clean = assign_description(pending_clean)
pending_clean

,timestamp,src_ip,src_port,session,node,description
0,2024-08-29 20:20:03.932693+00:00,162.241.71.207,NaN,3bb9bf322735,cd ~; chattr -ia .ssh; lockr -ia .ssh,Lock SSH directory
1,2024-08-29 20:20:04.094957+00:00,162.241.71.207,NaN,3bb9bf322735,"cd ~ && rm -rf .ssh && mkdir .ssh && echo ""ssh...",Save SSH key
2,2024-08-29 20:20:06.672065+00:00,162.241.71.207,NaN,3bb9bf322735,cat /proc/cpuinfo | grep name | wc -l,Get CPU counts
3,2024-08-29 20:20:06.781820+00:00,162.241.71.207,NaN,3bb9bf322735,"echo -e ""Abcd1234\nVuK2HnSyoq7D\nVuK2HnSyoq7D""...",change password
4,2024-08-29 20:20:06.782668+00:00,162.241.71.207,NaN,3bb9bf322735,Enter new UNIX password:,password prompt
...,...,...,...,...,...,...
79942,2024-09-25 04:22:02.039624+00:00,35.199.95.142,NaN,10ca21b91dfa,uname,Get system info
79943,2024-09-25 04:22:02.536076+00:00,35.199.95.142,NaN,10ca21b91dfa,uname -a,Get system info
79944,2024-09-25 04:22:03.095035+00:00,35.199.95.142,NaN,10ca21b91dfa,whoami,Get current username
79945,2024-09-25 04:22:03.563059+00:00,35.199.95.142,NaN,10ca21b91dfa,lscpu | grep Model,Get CPU model


In [23]:
csh_clean = assign_description(csh_clean)
csh_clean

,timestamp,src_ip,src_port,session,node,description
0,2024-08-29 00:21:01.725998+00:00,74.225.198.160,NaN,150d8a9ec37f,cd ~; chattr -ia .ssh; lockr -ia .ssh,Lock SSH directory
1,2024-08-29 00:21:02.363697+00:00,74.225.198.160,NaN,150d8a9ec37f,"cd ~ && rm -rf .ssh && mkdir .ssh && echo ""ssh...",Save SSH key
2,2024-08-29 00:21:06.451904+00:00,150.95.82.98,NaN,e3c01ed363e1,cd ~; chattr -ia .ssh; lockr -ia .ssh,Lock SSH directory
3,2024-08-29 00:21:07.319840+00:00,150.95.82.98,NaN,e3c01ed363e1,"cd ~ && rm -rf .ssh && mkdir .ssh && echo ""ssh...",Save SSH key
4,2024-08-29 00:21:07.476066+00:00,74.225.198.160,NaN,150d8a9ec37f,cat /proc/cpuinfo | grep name | wc -l,Get CPU counts
...,...,...,...,...,...,...
35162,2024-09-26 23:45:04.671393+00:00,192.166.123.50,NaN,5829c412f327,uname,Get system info
35163,2024-09-26 23:45:05.231464+00:00,192.166.123.50,NaN,5829c412f327,uname -a,Get system info
35164,2024-09-26 23:45:05.750555+00:00,192.166.123.50,NaN,5829c412f327,whoami,Get current username
35165,2024-09-26 23:45:06.179137+00:00,192.166.123.50,NaN,5829c412f327,lscpu | grep Model,Get CPU model


In [24]:
ebilling_clean = assign_description(ebilling_clean)
ebilling_clean

,timestamp,src_ip,src_port,session,node,description
0,2024-08-29 21:56:13.230854+00:00,5.182.211.148,NaN,95542304d741,chmod +x clean.sh; sh clean.sh; rm -rf clean.s...,"run setup script, create SSH key"
1,2024-08-29 23:02:34.832484+00:00,14.58.14.151,NaN,24ca59a2655b,/ip cloud print,OS info for a router
2,2024-08-29 23:02:35.439614+00:00,14.58.14.151,NaN,24ca59a2655b,ifconfig,Get network interface
3,2024-08-29 23:02:36.088560+00:00,14.58.14.151,NaN,24ca59a2655b,uname -a,Get system info
4,2024-08-29 23:02:36.748122+00:00,14.58.14.151,NaN,24ca59a2655b,cat /proc/cpuinfo,Get CPU info
...,...,...,...,...,...,...
36590,2024-09-25 17:15:39.162929+00:00,179.104.32.50,NaN,4ea0b47f5c5f,uname,Get system info
36591,2024-09-25 17:15:39.665726+00:00,179.104.32.50,NaN,4ea0b47f5c5f,uname -a,Get system info
36592,2024-09-25 17:15:40.239858+00:00,179.104.32.50,NaN,4ea0b47f5c5f,whoami,Get current username
36593,2024-09-25 17:15:40.832798+00:00,179.104.32.50,NaN,4ea0b47f5c5f,lscpu | grep Model,Get CPU model


In [25]:
pending_clean.to_csv('cleaned_csv/pending_clean.csv')
ebilling_clean.to_csv('cleaned_csv/ebilling_clean.csv')
csh_clean.to_csv('cleaned_csv/csh_clean.csv')
